In [ ]:
#새로운 회사 크롤링 모듈
"""
새로운 회사 리스트를 받아서
1.기업정보 크롤링 요청 및 테이블 생성
2.뉴스정보 크롤링 요청및 테이블 생성
3.채용 공고도 크롤링
"""

In [1]:
import os
os.getcwd()

'/app/EZPZ_Project/scheduler'

In [3]:
import sys
import os
import pandas as pd


sys.path.append('/app/EZPZ_Project/modules/crawlers/news') # 뉴스 정보 크롤러 경로
sys.path.append('/app/EZPZ_Project/modules/torchmodules') # 토치 모델 뉴스 요약 및 감정평가 가져오기
sys.path.append('/app/EZPZ_Project/modules/crawlers/job_post') #채용공고 크롤러
sys.path.append('/app/EZPZ_Project/modules/crawlers/comp_info') #기업 정보 크롤러
sys.path.append('/app/EZPZ_Project') #db 연동정보 경로


In [4]:

import pymysql
import socket
import cryptography

from service_models import ServiceModels
from finbertkr import FinBertKR
from mt5sum import MT5Sum
from t5basesum import T5BaseSum



import daum_news_crawler #다음뉴스크롤러
import naver_news_crawler #네이버 뉴스크롤러
import news_crawlers #네이버 뉴스 크롤러2
import wanted_recruit_crawler #채용공고 크롤러
import info_crawler #기업정보 크롤러


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


   recruit_uid                         recruit_url  \
0     {174812}  https://www.wanted.co.kr/wd/174812   
1     {174746}  https://www.wanted.co.kr/wd/174746   
2     {172571}  https://www.wanted.co.kr/wd/172571   
3     {172570}  https://www.wanted.co.kr/wd/172570   
4     {172557}  https://www.wanted.co.kr/wd/172557   
5     {174460}  https://www.wanted.co.kr/wd/174460   
6     {172569}  https://www.wanted.co.kr/wd/172569   
7     {174452}  https://www.wanted.co.kr/wd/174452   
8     {139818}  https://www.wanted.co.kr/wd/139818   
9     {151080}  https://www.wanted.co.kr/wd/151080   
10    {176814}  https://www.wanted.co.kr/wd/176814   
11    {175906}  https://www.wanted.co.kr/wd/175906   

                               recruit_position  \
0                     헬스케어 모바일 서비스 기획자 (1년 계약직)   
1                               헬스케어 데이터 사이언티스트   
2                                  Back-end 개발자   
3                                   Android 개발자   
4                                  정보유출 모니

In [5]:

from privates.ezpz_db import * #db연동 정보

In [6]:
import sql_connection as sc

In [15]:
comp_list = []
sql = ' select comp_name from comp_info where is_reged = "N" '
comp_temp_list = sc.conn_and_exec(sql)
for comp in comp_temp_list:
    comp_list.append(comp[0])
#return comp_list #리스트 받아와서 is reged y 회사마다 바꿔주고 modify_date만 바꿔주면됨

In [16]:
comp_list

['세이지리서치(주)',
 '(주)골라라',
 '(주)하룹',
 '(유)밸러스',
 '페이스북코리아(유)',
 '(사)금융보안원',
 '애터미(주)',
 '(주)핫셀러',
 '라이엇게임즈코리아',
 '(주)익스피디아코리아',
 '(주)케이에스에스해운',
 '경동도시가스(주)',
 '(주)페이타랩',
 '브라운백(주)',
 '구글코리아(유)',
 '네이버웹툰',
 '맥킨지인코포레이티드한국지점',
 '(주)넥슨게임즈',
 '(주)트레드링스',
 '한국중부발전(주)',
 '(주)센드버드코리아',
 '매스웍스코리아(유)',
 '스캐터랩',
 '한국남동발전(주)',
 '한국증권금융(주)',
 '로버스트자산운용(주)',
 '(주)버핏서울',
 '한국노동연구원',
 '유니티테크놀로지스코리아(유)',
 '한국가스공사',
 '(유)한국어도비시스템즈',
 '(주)베이글코드',
 '(주)삼성글로벌리서치',
 '한국예탁결제원',
 '(주)에스피에이치',
 '에스케이종합화학(주)',
 '(주)큐픽스',
 '인천도시공사',
 '(주)유니메오',
 '삼성인력개발원',
 '한국천문연구원',
 '행복쇼핑(주)',
 '(주)디벨록',
 '부산교통공사',
 '구 카카오(주)',
 '네이버제트(주)',
 '베인앤드컴퍼니코리아인크(주)',
 '워터스코리아(유)',
 '(주)벡터코리아아이티',
 '(주)비츠로밀텍',
 '(주)데이블',
 '서울축산업협동조합',
 '인천국제공항공사',
 '한국해양진흥공사',
 '(재)한국원자력연구원',
 '클립소프트(주)',
 '한국동서발전(주)',
 '한국무역보험공사',
 '한국전자통신연구원',
 '(주)슬라운드',
 '(주)에이스프로젝트',
 '주한미국대사관',
 '에스에이피랩스코리아(주)',
 '지쿱(주)',
 '한국퀄컴(주)',
 '(사)금융결제원',
 '(주)한국거래소',
 '기술보증기금',
 '한국은행',
 '(주)그립컴퍼니',
 '(주)살다',
 '이넵스(주)',
 '한국서부발전(주)',
 '한국주택금융공사',
 '(재)한국지질자원연구원',
 '(주)킨다

In [20]:
comp_info_df=info_crawler.get_url('지쿱')
comp_info_df['comp_cont']=comp_info_df['comp_cont'].iloc[0][0]
comp_info_df


,comp_uid,comp_name,comp_loc,comp_thumb,comp_cont,comp_founded,comp_size,comp_url
,354866,지쿱(주),전북 남원시 밤티재길 16-12,https://jpassets.jobplanet.co.kr/production/up...,판매유통,2015.05.08,-,http://www.gcoop.com


In [7]:
sql = f'select comp_uid from comp_info where replace(comp_name , " ", "") like "%제너럴바이오(주)%" '
uid = sc.conn_and_exec(sql)
comp_uid= uid[0][0]
comp_uid

64

In [3]:
conn = get_connection()
cur = conn.cursor()



In [18]:

data_check= ServiceModels() #모델 서빙 모듈 객체

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:
cur.execute('desc comp_news') #key연결 순서대로 다 drop 해버리기

9

In [40]:
cur.execute('select * from comp_news')

0

In [41]:
cur.execute('select * from comp_info')
for i in cur:
    print(i)

(84, '삼성전자(주)', '경기 수원시 영통구 삼성로 129 (매탄동,주차타워)', 'https://jpassets.jobplanet.co.kr/production/uploads/company/logo/30139/thumb_Operation_Logo_guide.jpg', '전기/전자/제어', '196901', '대기업', 'http://www.samsung.com/sec/', 'Y', '000000', '000000')
(85, '(주)카카오', '제주 제주시 첨단로 242', 'https://jpassets.jobplanet.co.kr/production/uploads/company/logo/93880/thumb_kakao.jpg', '포털/인터넷/콘텐츠', '199502', '중견기업', 'http://www.kakaocorp.com', 'Y', '000000', '000000')
(86, '네이버(주)', '경기 성남시 분당구 불정로 6 (정자동,그린팩토리)', 'https://jpassets.jobplanet.co.kr/production/uploads/company/logo/42217/thumb______.jpg', '포털/인터넷/콘텐츠', '199906', '중견기업', 'https://www.navercorp.com/', 'Y', '000000', '000000')


In [42]:
cur.execute('select * from recruit_info')
for i in cur:
    print(i)

(39622, 86, 'https://www.wanted.co.kr/wd/39622', '카카오/네이버 연계 시스템 서버 개발자', 'https://static.wanted.co.kr/images/company/3342/nmshwilloxvcqlxq__1080_790.png', '00000000', '00000000')
(149197, 85, 'https://www.wanted.co.kr/wd/149197', 'Inference Engine Engineer', 'https://static.wanted.co.kr/images/company/22504/u2ryao0fwotejf3h__1080_790.jpg', '00000000', '00000000')
(151828, 85, 'https://www.wanted.co.kr/wd/151828', 'Applied AI/ML Engineer', 'https://static.wanted.co.kr/images/company/22504/u2ryao0fwotejf3h__1080_790.jpg', '00000000', '00000000')
(156751, 85, 'https://www.wanted.co.kr/wd/156751', 'Biology AI Researcher', 'https://static.wanted.co.kr/images/company/22504/u2ryao0fwotejf3h__1080_790.jpg', '00000000', '00000000')
(159350, 85, 'https://www.wanted.co.kr/wd/159350', 'Backend Engineer(AI Image Service)', 'https://static.wanted.co.kr/images/company/22504/u2ryao0fwotejf3h__1080_790.jpg', '00000000', '00000000')
(169809, 86, 'https://www.wanted.co.kr/wd/169809', '네트워크 엔지니어 (춘천 네이버I

In [29]:
cur.execute('select * from comp_news')
for i in cur:
    print(i)

(57, 84, '20230817', 'http://v.daum.net/v/20230817070650924', '[서울=뉴시스]이인준 기자 = 메모리 업황 회복 기대감으로 올 2분기(4~6월) 삼성전자 주가가 7만원대를 회복하자, 일부 임원들이 주식을 매각해 차익 실현에 나선 것으로 나타났다. 이에 따라 삼성전자 임원들의 올 2분기 말 보유 주식 총수는 전 분기 대비 마이너스를 보였다. 16일 금융감독원 전자공시에 따르면 삼성전자 미등기임원이 보유한 보통주는 지난 6월 말 기준 9815만6000주로, 지난 3월 말 9816만4979주 대비 8979주 감소했다. 여기서 이재용 회장 보유 지분(9741만4196주)를 빼면 미등기 임원의 순수 보유 주식은 75만783주에서 74만1804주로 줄어든다. 삼성전자 미등기임원 보유 주식 수가 이렇게 감소한 것은 올 들어 처음이다. 이와 함께 의결권이 없는 우선주도 같은 기간인 석달 동안 17만4559주에서 17만3245주로 1314주 감소했다. 우선주 역시 주가가 오르자 일부 임원들이 차익 실현을 했기 때문이다. 삼성전자 사업보고서에 따르면 보통주 평균가격은 지난해 9월 5만5595원까지 내렸고, 이후 6만원 초반대에서 횡보하는 모습을 보였다. 이에 삼성전자 임원들은 당시 회사 주식을 집중 매입한 것으로 알려졌다. 횡보하던 주가는 올 하반기에 업황이 바닥을 찍을 것이라는 기대감이 커지자, 올 2분기(4~6월)에 상승세를 보였다. 특히 지난 6월 평균가격은 7만1676원으로 7만원대를 회복하기도 했다. 이에 미등기 임원들은 일부 주식을 2분기에 매각한 것으로 추정된다. 전자공시시스템에 따르면 2분기에 삼성전자 등기임원이 회사 주식을 매입하거나 매각한 사례는 없었다.이와 달리 삼성전자를 향한 소액주주들의 애정 공세는 여전히 계속됐다. 올 상반기 말 기준 소액주주가 보유한 삼성전자 주식 수는 40억537만주로, 지난해 12월 말 39억9208만주 대비 1329만주(0.3%) 증가했다. 발행주식 전체에서 소액주주 주식이 차지하는

In [15]:
comp_list= ['삼성전자(주)','(주)카카오','네이버(주)']

In [20]:
cur.execute('delete from comp_info') #언제든지 삭제가능 위치 뒤바꿔야 될수도 있음
cur.execute('delete from comp_news')
cur.execute('delete from recruit_info')

0

In [13]:
conn.commit()

In [42]:
cur.execute('select * from recruit_info order by comp_uid')
for i in cur:
    print(i)

: 

In [36]:
crawler_exec(comp_list)

실행시작
삼성전자
56
카카오
57
네이버
58


In [32]:
#여기서 함수를 실행해서 각종 정보들 실행
def crawler_exec(comp_list):
    comp_info_crawl_save(comp_list)
    #comp_news_crawl_save(comp_list)
    recruit_info_crawl(comp_list)
    


In [11]:
#먼저 comp_info 크롤링해와서 테이블에 넣어줘야겠지? 근데 no니까 is_reged 전부 y로 바꿔주고 나중에 테이블 전부다
def comp_info_crawl_save(comp_list):
    for comp in comp_list:
        comp_info_df=info_crawler.get_url(comp) #info_crawler 기업정보 데이터 프레임 잘 어울림
        # 기업 정보 리스트로 저장해서 데이터프레임에 넣어줄수 있게
        col_value=[]
        for i in comp_info_df.iloc[0]:
            col_value.append(i) #테이블에 insert할 수 있는 컬럼값들을 리스트화


        col_value[5]=col_value[5][0:7]
        col_value[5] = col_value[5].replace(".", "")
        col_value[5] # 6글자 문자열로 변환 테이블에 형식대로

        #크롤링해온 값 테이블에 저장 저장일자와 수정일자는 스케줄링단계에서 진행이므로 일단 000000 넣어두었음
        #일단 testdb를 경로로했는데 나중에 바꿔줘야됨
        cur.execute(f"INSERT INTO testdb.comp_info (comp_name, comp_loc, comp_thumb, comp_cont, comp_founded, comp_size, comp_url, is_reged, create_date, modify_date) VALUES (%s, %s, %s, %s, %s, %s, %s, 'N', '000000', '000000')", (col_value[1], col_value[2], col_value[3], col_value[4], col_value[5], col_value[6], col_value[7]))      
        
    #값들을 전부 넣어줬으니 update y로
    cur.execute('UPDATE comp_info SET is_reged = "Y" ')

In [65]:

#얘 잘되는지는 .py 파일에서 확인
def comp_news_crawl_save(comp_list):
        print('실행시작')
        for comp in comp_list:
            daum_news = daum_news_crawler.get_news(comp).head(1) #다음뉴스 크롤러 실행 확인
            naver_news = naver_news_crawler.get_news(comp).head(1) #네이버 뉴스크롤러 실행
            all_news = pd.concat([daum_news, naver_news], ignore_index=True)  #뉴스 전체 합치기
            print(all_news)
            for index, col in enumerate(all_news['news_cont']):
                if len(col)>5000:
                    all_news['news_cont'].iloc[index] = col[:5000] #5000자 이상은 cut이므로 이걸로 체크

            #모델 돌리기
            cont_sum_list= [] #df에 넣어줄 요약 리스트
            cont_sent_list=[] # df에 넣어줄 감정평가 리스트
            senti_to_int=[] # 감정을 정수형으로 바꿔줄 리스트

            for text in all_news['news_cont']:
                cont_sum=data_check.get_summary(text, 'news')
                cont_sum_list.append(cont_sum)
                
            
            for text in cont_sum_list:
                cont_sent=data_check.get_sentiment(text)
                cont_sent_list.append(cont_sent)
                # df_news_senti에 값을 0(중립), 1(긍정), 2(부정)으로 바꿔줘야함
            
            for col in (cont_sent_list):
                if col =='neutral':
                    senti_to_int.append(0)
                elif col =='positive':
                    senti_to_int.append(1)
                else:
                    senti_to_int.append(2)
            
            #데이터프레임에 요약 결과와 감정평가 결과 넣어주기
            all_news['news_sum'] = cont_sum_list
            all_news['news_senti'] = senti_to_int

            for index,col in enumerate(all_news['news_sum']):
                if len(col)>256:
                    all_news['news_sum'].iloc[index] = col[:256] #5000자 이상은 cut이므로 이걸로 체크

            #기사 내용과 요약본에 따옴표들 전부 삭제 전처리
            clean_cont=[]
            clean_sum=[]
            for i in all_news['news_cont']:
                
                cont_clean = i.replace('"', '').replace("'", '')
                clean_cont.append(cont_clean)
                


            for j in all_news['news_sum']:
                sum_clean = j.replace('"', '').replace("'", '')
                clean_sum.append(sum_clean)

            all_news['news_cont']= clean_cont
            all_news['news_sum'] = clean_sum         
            
            get_comp_news_db(all_news,comp) #실행될때마다 바뀌는 기업별 all_news 테이블화 시키기

    

In [66]:
def get_comp_news_db(all_news,comp): # 만들어진 데이터프레임을 테이블로
    cur.execute(f'select comp_uid from comp_info where comp_name = "{comp}"') 
    comp_uid=cur.fetchall()[0][0]
    # news_uid는 auto increment니까 자동생성되지 않을까?
    for index, row in all_news.iterrows():
            sql = 'insert into comp_news '
            sql += '    (comp_uid, pub_date, news_url, news_cont,news_sum, news_senti, create_date, modify_date) '
            sql += 'values ( '
            sql += f'   "{comp_uid}", "{row["pub_date"]}", "{row["news_url"]}", "{row["news_cont"]}", "{row["news_sum"]}", "{row["news_senti"]}" '
            sql += f'    , "{"00000000"}", "{"00000000"}" '
            sql += ') '
            cur.execute(sql)

In [25]:
def recruit_info_crawl(comp_list):
    print('실행시작')
    for comp in comp_list:
        #채용공고는 주 붙어있으면 안됨 제거 전처리
        recruit_comp = comp.replace('(주)',"")
        print(recruit_comp)
        recruit_info_df=wanted_recruit_crawler.get_recruit_info(recruit_comp, csv_save=False) # 원티드 기업정보 크롤러 모듈
        new_i=[] #집합인 uid를 int로 바꿔준 값을 넣어준 리스트/ 테이블에 넣기위한 전처리
        for i in range(len(recruit_info_df['recruit_uid'])):
            new_i.append(list(recruit_info_df['recruit_uid'][i])[0])
        recruit_info_df['recruit_uid']=new_i #int 값으로 컬럼 대체


        cur.execute(f'select comp_uid from comp_info where comp_name = "{comp}"') 
        comp_uid=cur.fetchall()[0][0]
        print(comp_uid)
        for index, row in recruit_info_df.iterrows():
            sql = 'insert into recruit_info '
            sql += '    (comp_uid, recruit_uid, recruit_url, recruit_position, recruit_thumb, create_date, modify_date) '
            sql += 'values ( '
            sql += f'   "{comp_uid}", "{row["recruit_uid"]}", "{row["recruit_url"]}", "{row["recruit_position"]}", "{row["recruit_thumb"]}" '
            sql += f'    , "{"00000000"}", "{"00000000"}" '
            sql += ') '
            cur.execute(sql)
            for i in cur:
                print(i)

In [43]:
conn.close()